In [1]:
import tensorflow as tf
import numpy as np
import utils

from collections import Counter


__author__ = "Olivares Castillo José Luis"

#tf.enable_eager_execution()
tf.logging.set_verbosity(tf.logging.INFO)

print("TensorFlow version: {}".format(tf.VERSION))
#print("Eager execution: {}".format(tf.executing_eagerly()))

if tf.test.gpu_device_name():
    print("GPU disponible")

/home/olivares/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow version: 1.7.0


# Cargar lexicones

In [26]:
src_lex, trg_lex = utils.get_lexicon("en-it.train")
print(len(set(src_lex)), len(set(trg_lex)))

3442 4549


#### Ejecutar celda sólo cuando se utiliza el lexicon de pruebas.

In [3]:
src_lex = list(set(src_lex))

# Obtener vectores de entrenamiento/prueba

* Se cuentan las palabras en el lexicon con la función `Counter()`
* Se abre el archivo de vectores.
* Se obtienen los vectores de las palabras en el lexicon y se guardan en una matrix

In [29]:
%%time
src_lex = Counter(src_lex)

src_matrix = utils.open_file("en-wiki")

words_src, source_vec = utils.read(src_matrix,vocabulary=src_lex)

print(source_vec.shape)

(5000, 300)
CPU times: user 43.1 s, sys: 2.36 s, total: 45.5 s
Wall time: 48.5 s


### Se carga la matrix de vectores del lenguaje objetivo.

In [45]:
%%time
trg_matrix = utils.open_file("it")

words_trg,target_vec = utils.read(trg_matrix)

print(target_vec.shape)

(2000000, 300)
CPU times: user 2min 17s, sys: 3.56 s, total: 2min 21s
Wall time: 2min 53s


# Leer modelo desde archivo

* Se reinicia el grafo cada que se lee un nuevo modelo.
* Se inicia una sesión de `TensorFlow`.
* Se lee el modelo y se carga el grafo.

In [31]:
tf.reset_default_graph()
sess = tf.Session()
path="models/1/"
saver = tf.train.import_meta_graph(path+"model.ckpt.meta")
saver.restore(sess,tf.train.latest_checkpoint(path))

INFO:tensorflow:Restoring parameters from models/1/model.ckpt


# Obtener tensores de entrada
* Se obtiene el grafo y se guardan en variables los tensores de entrada para alimentar al modelo.

In [32]:
graph = tf.get_default_graph()
X = graph.get_tensor_by_name("input/input_es:0")
kprob = graph.get_tensor_by_name("dropout_prob:0")

In [33]:
#([n.name for n in graph.as_graph_def().node])

# Se obtiene la salida del modelo
* Se guarda en una variable la operación que se va a ejecutar.
* Se indican las entradas del modelo en `feed_dict`.
* Se ejecuta la operación y se obtiene guarda el resultado en `pred` que son los vectores transformados por el modelo.

In [34]:
output_model = graph.get_tensor_by_name("xw_plus_b_1:0")
#output_model = graph.get_tensor_by_name("nah_predicted:0")
feed_dict = {X: source_vec, kprob: 1}
pred = sess.run(output_model, feed_dict)
print(pred.shape)

(5000, 300)


In [43]:
np.sum(np.abs(pred[0]-source_vec[0]))

28.546037468132205

UsageError: Line magic function `%memory` not found.
